## questions I'm curious about

- am I listing to full albums?
- what decade do i listen to most?
- what are my most listened to genres?
- what are my most listened to albums?
- are songs I'm listening to on my playlists or in my library?
- can I tell someones mood that day based on what they are listening to?

## Info I want about every song

- Track ID
- Track Name
- Artists ID
- Artist Name
- Length Of Track
- Genres List of Track
- End Time of Stream
- Time of Stream
- % of Song Listened To
- Is Explicit
- Disc Number

## Things I've done so far

- Spotipy authorization and scope.
    - Scope defines what data you're allow to pull from that API request
    - I set environment variables of my client and secret needed for at least some authorization
    - Does all authorization require scope?
- Spotify Data Request
    - They only retain 1 years worth of streaming data
- 

# Initialize

In [1]:
import json
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials,SpotifyOAuth
import pandas as pd
from datetime import date, datetime
import time
from dateutil import tz

## Pick your auth version

In [2]:
user_id = 'brian.cross741'
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)
#sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

# Basic Functions

In [3]:
# opens json files provided by spotify data retreival
def get_json_from_file(spotify_file):
    f = open('MyData/{}.json'.format(spotify_file), encoding="utf8")
    data = json.load(f)
    f.close()
    return data

# # searches for any string and returns the artist_id for the top response item
# def search_for_artist_id(name):
#     result = sp.search(name)
#     id = result['tracks']['items'][0]['artists'][0]['id']
#     return id

# # searches for any string and returns the track_id for the top response item
# def search_for_track_id(name):
#     result = sp.search(name)
#     id = result['tracks']['items'][0]['id']
#     return id

# searches for any string and returns the track_id for the top response item
# def search_for_track(track_name=None, artist_name=None):
#     search_result = sp.search(q='track:{} artist:{}'.format(track_name,artist_name), type='track')
#     subset = parse_track_result(search_result['tracks']['items'][0])    
#     return subset


def parse_episode_result(item):
    subset = dict((k, item[k]) for k in ('id', 'name', 'duration_ms', 'explicit', 'type'))
    return subset

def search_for_track(track_name=None, artist_name=None):
    try:
        search_result = sp.search(q='''track:{} artist:{}'''.format(track_name,artist_name), type='track')
        subset = parse_track_result(search_result['tracks']['items'][0])
    except:
        try:
            search_result = sp.search(q='''episode:{}'''.format(track_name), type='episode')
            subset = parse_episode_result(search_result['episodes']['items'][0])
            subset['artist_name'] = artist_name
        except:
            subset = {'artist_name':artist_name, 'track_name':track_name}
    return subset



def get_artist_data(id):
    result = sp.artist(id)
    subset = dict((k, result[k]) for k in ('id', 'name', 'genres', 'popularity'))
    subset['followers'] = result['followers']['total']
    return subset

def get_track_data(id):
    result = sp.track(id)
    subset = dict((k, result[k]) for k in ('id', 'name', 'duration_ms', 'popularity', 'disc_number', 'track_number', 'explicit', 'type'))
    subset['album_id'] = result['album']['id']
    subset['artist_id'] = result['artists'][0]['id']
    subset['artist_count'] = len(result['artists'])
    return subset

def get_album_data(id):
    result = sp.album(id)
    subset = dict((k, result[k]) for k in ('id', 'name', 'album_type', 'release_date', 'release_date_precision', 'popularity', 'total_tracks', 'genres', 'label', 'type'))
    return subset
    

# Recently Played Functions

In [4]:

###
# opens all the recently palyed files and finds the most recent timestamp, converting it to local time
# returns unix timestamp and a datetime for the most recently played track in the logs
###
def get_most_recent_timestamp():
    # folder containing saved files of recently played
    path = 'recently_played/'
    recently_played_files = os.listdir(path)

    total_max_played = ''
    total_max_played_unix = 0
    for i in recently_played_files:
        current_max_played = pd.read_csv(path+i)['played_at'].max()
        utc = datetime.strptime(current_max_played, "%Y-%m-%dT%H:%M:%S.%fZ")
        current_max_played_unix = int(time.mktime(utc.timetuple())*1000)
    
        if current_max_played_unix > total_max_played_unix:
            total_max_played_unix = current_max_played_unix
            total_max_played = utc
    
    from_zone = tz.tzutc()
    to_zone = tz.tzlocal()
    total_max_played = total_max_played.replace(tzinfo=from_zone)
    local_total_max_played = total_max_played.astimezone(to_zone)
    local_total_max_played_unix = int(time.mktime(local_total_max_played.timetuple())*1000)
    return local_total_max_played_unix, local_total_max_played

###
# runs through the results of recently played and flattens the dictionary
# returns a cleaned dictionary of recently played
###
def parse_track_result(item):
    try:
        result = item['track']
    except:
        result = item
    subset = dict((k, result[k]) for k in ('id', 'name', 'duration_ms', 'popularity', 'disc_number', 'track_number', 'explicit', 'type'))
    subset['album_id'] = result['album']['id']
    subset['artist_id'] = result['artists'][0]['id']
    subset['artist_count'] = len(result['artists'])
    try:
        subset['context'] = item['context']['uri']
    except:
        pass
    try:
        subset['played_at'] = item['played_at']
    except:
        pass
    return subset

###
# intakes either None or a unix timestamp which will call all records occurring after it
# returns a cleaned dictionary of recently played
###
def call_recently_played(after_timestamp_unix):
    # set permissions, the scope is needed for the API to be allowed to grab this data.
    scope = "user-read-recently-played"
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
    
    # call spotify
    recently_played = sp.current_user_recently_played(limit=50, after=after_timestamp_unix)

    # parse data and return a shorter, flattened dictionary with track data
    recently_played_short = []
    for i in recently_played['items']:
        subset = parse_track_result(i)
        recently_played_short.append(subset)
    return recently_played_short

###
# runs through the recently played directory, finds the most recent timestamp, and calls recently played for the new, unsaved records
# returns a cleaned dataframe of non-recorded recently played
###
def get_new_recently_played():
    after_timestamp = get_most_recent_timestamp()[0]
    recently_played = call_recently_played(after_timestamp+1000)
    df = pd.DataFrame(data=recently_played)
    return df

###
# intakes a dataframe and savesit as a log in the recently played folder
# tells you the timestamp used to save
###
def save_new_recently_played(df):
    dt_string = datetime.now().strftime("%Y%m%d_%H_%M_%S")
    if len(df) > 0:
        df.to_csv(f'recently_played/recently_played_{dt_string}.csv', index=False)
        print('saved file: {}'.format(dt_string))

### Save Recently Played

In [6]:
get_most_recent_timestamp()

(1697629141000,
 datetime.datetime(2023, 10, 18, 7, 39, 1, 795000, tzinfo=tzlocal()))

In [7]:
scope = "user-read-recently-played"
# scope = 'user-read-playback-state'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
recently_played = sp.current_user_recently_played(limit=50, after=get_most_recent_timestamp()[0]+10000)

In [8]:
recently_played['items'][0]['context']

In [10]:
new_recently_played = get_new_recently_played()

In [11]:
len(new_recently_played)

50

In [13]:
new_recently_played.tail()

,id,name,duration_ms,popularity,disc_number,track_number,explicit,type,album_id,artist_id,artist_count,played_at
45,5VBo8z9x2dK1C5TB634LwA,Mambo No.8,128133,18,1,7,False,track,4gIaN0sSqN7fjDq9NG4vFp,1ZKhPkCXXgtiGgALn4OYtT,1,2023-10-18T13:20:50.530Z
46,66ND8DY2EczHltZ2snI3qW,Taurus In The Arena Of Life,257200,25,1,4,False,track,03zH4bkGb2jSZAQIVW2px1,1W8TbFzNS15VwsempfY12H,1,2023-10-18T13:15:14.797Z
47,6pCQ5u1P1GGOXllx6H4FhY,"Don't Be Afraid, The Clown's Afraid Too",565466,25,1,3,False,track,03zH4bkGb2jSZAQIVW2px1,1W8TbFzNS15VwsempfY12H,1,2023-10-18T12:45:34.697Z
48,57EMqntPq6sptkSbc5lyKi,Adagio Ma Non Troppo,503440,28,1,2,False,track,03zH4bkGb2jSZAQIVW2px1,1W8TbFzNS15VwsempfY12H,1,2023-10-18T12:35:44.582Z
49,3fcKZiegoVsBmVuGkyvcWv,The Shoes of the Fisherman's Wife Are Some Jiv...,574426,37,1,1,False,track,03zH4bkGb2jSZAQIVW2px1,1W8TbFzNS15VwsempfY12H,1,2023-10-18T12:27:21.623Z


In [14]:
save_new_recently_played(new_recently_played)

saved file: 20231018_19_06_47


# Expand Streaming Data

In [131]:
# search_result = sp.search(q='episode:{}'.format('MBMBaM 624: Reince Your Own Priebus'), type='episode')

In [15]:
streaming_data = get_json_from_file('StreamingHistory0')

In [16]:
len(streaming_data)

7886

### Searching to expand streaming data

In [181]:
# honestly this is really sloppy and the iterating and saving part should be overhauled.

result_list = []
iteration = 5000
start = datetime.now()
for track in streaming_data[5000:]:
    iteration += 1
    artist_name = track['artistName']
    track_name = track['trackName']
    end_time = track['endTime']
    ms_played = track['msPlayed']

    # search spotify using the track and artist name
    result = search_for_track(track_name=track_name, artist_name=artist_name)
    result['duration_ms'] = ms_played
    result['played_at'] = end_time
    result_list.append(result)
    if iteration % 500 == 0:
        print('saving {}'.format(iteration))
        df_streaming_data = pd.DataFrame(data=result_list)
        df_streaming_data.to_csv('searched_history/raw_searches/streaming_data_searched_{}.csv'.format(iteration+1))
    else:
        pass
end = datetime.now()
print(end - start)
df_streaming_data = pd.DataFrame(data=result_list)
df_streaming_data.to_csv('searched_history/raw_searches/streaming_data_searched_{}.csv'.format(iteration+1))

saving 5500
saving 6000
saving 6500
saving 7000
saving 7500
0:13:28.306981


### Combining streaming data files into on

In [196]:
path = "searched_history"
dir_list = os.listdir(path)
print(dir_list)

stream_df = pd.DataFrame()

for i in dir_list:
    read_file = pd.read_csv(path+'/'+i)
    read_file.drop('Unnamed: 0',axis=1,inplace=True)
    stream_df = pd.concat([stream_df,read_file])
stream_df.drop_duplicates(inplace=True)

['streaming_data_searched_1000.csv', 'streaming_data_searched_1500.csv', 'streaming_data_searched_2000.csv', 'streaming_data_searched_2500.csv', 'streaming_data_searched_3000.csv', 'streaming_data_searched_3500.csv', 'streaming_data_searched_4000.csv', 'streaming_data_searched_4500.csv', 'streaming_data_searched_500.csv', 'streaming_data_searched_5000.csv', 'streaming_data_searched_5501.csv', 'streaming_data_searched_6001.csv', 'streaming_data_searched_6501.csv', 'streaming_data_searched_7001.csv', 'streaming_data_searched_7501.csv', 'streaming_data_searched_7887.csv']


In [211]:
stream_tracks = stream_df[stream_df['type'] == 'track']
stream_episodes = stream_df[stream_df['type'] == 'episode']
stream_errors = stream_df[stream_df['type'].isnull()]

In [214]:
stream_tracks.to_csv('searched_history/streaming_history_tracks.csv',index=False)
stream_episodes.to_csv('searched_history/streaming_history_episodes.csv',index=False)
stream_errors.to_csv('searched_history/streaming_history_errors.csv',index=False)

# Track Object

### Helpers

In [ ]:
subset = dict((k, result[k]) for k in ('id', 'name', 'duration_ms', 'popularity', 'disc_number', 'track_number', 'explicit', 'type'))


result['album']
result['album']['album_type']
result['album']['id']
result['album']['name']
result['album']['release_date']
result['album']['release_date_precision']
result['album']['total_tracks']
result['album']['type']

result['artists']
result['artists'][artist_order]['name']
result['artists'][artist_order]['id']
result['artists'][artist_order]['type']

result['disc_number']
result['track_number']
result['duration_ms']
result['explicit']
result['id']
result['name']
result['popularity']
result['type']

## Class

In [97]:
class track:
    def __init__(self, id):
        self.id = id
        self.data = sp.track(id)
        self.album = sp.album(self.data['album']['id'])

    def __repr__(self):
        return '"{}" by {}'.format(self.data['name'],self.data['artists'][0]['name'])


__Well what do I do with a class once I have it?__

I think maybe I should just work with the data in a single table?
But it's hard to because I don't know what the data model should be.

### Thoughts on Data Schema and Metrics

__Genre Issue:__
- Where does it come from?
    - Genre can come from artist or album
    - If it's on the album then get it from the album (album has genre but sometimes it's null)
    - If it's artist level genre, then how can you blend genres across artists?
        - Rank most common genre
        - Rank most common genre, favoring first artist
        - Rank most common genre, favoring first genre order
        - Filter for top 3 genres, match all of them
        - Generate list by taking top genre from each artist then using those as the genre list, or expand to include all genres.
        - If the genre orders match, then genre should remain in that order placement (I don't actually like that)
- How to narrow down from a single list of genres
    - Take first genre
    - Merge first and second genre
    - Second Genre (for hierarchy)
    - Third Genre (for hierarchy)
- Should I just look up genre from another source?
- I need to assign the genre to each track because artists have their own genres, separate from tracks and albums.

__Artist Issue:__
- Who even is the artist if there are multiple artists listed?
    - Take first artist (best for analytics)??
    - Merge first and second artist
    - Second Genre (for hierarchy)
    - Third Genre (for hierarchy)

__Metrics__

Well what do you want?
- streams over time
- most streamed artist
- most streamed album
- most streamed genre
- distribution of artist popularity
- distibution of track popularity
- genre streams over time
- streams
- which artists have the most diverse album genres
- most listened to album


__Issues with Data Model__

There seem to be so many different ways tom combine the following objects:
- stream
- track
- artist
- album
- playlist

Everything is relational, it depends on the question you are asking that determines the data needed.

The Spotify API tries to give enough data for what you need, but not give you everything.

__Data Model__
- stream
- track 1:many stream
- artist many:many track
- album 1:1 track
- playlist 1

__Alternate Data Model__
- album
- artist many:many album

### What do users do?
- They listen to tracks
- tracks have albums
- albums have artists
- tracks have an artist and a feature(s) when they are singles.
    - QA Note: Do the artists on single albums exactly match the artist on the track?
- tracks have collaborators when they aren't
- If the album is an anthology, the artist on the track won't match the artist on the album
    - QA Note: How often do track artists not match album artists
 
- Why does it matter what album the track is on?
    - Most streamed album metric
    - Did you stream the whole album in a row
    - What percentage of the album did you stream in a day

In [98]:
my_track = track('6JjSoJ2laV4ZbKNb9nybvh')

In [107]:
my_track.album#['genres']

{'album_type': 'single',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7tm9Tuc70geXOOyKhtZHIj'},
   'href': 'https://api.spotify.com/v1/artists/7tm9Tuc70geXOOyKhtZHIj',
   'id': '7tm9Tuc70geXOOyKhtZHIj',
   'name': 'Chris Lorenzo',
   'type': 'artist',
   'uri': 'spotify:artist:7tm9Tuc70geXOOyKhtZHIj'},
  {'external_urls': {'spotify': 'https://open.spotify.com/artist/1AHswQqsDNmu1xaE8KpBne'},
   'href': 'https://api.spotify.com/v1/artists/1AHswQqsDNmu1xaE8KpBne',
   'id': '1AHswQqsDNmu1xaE8KpBne',
   'name': 'COBRAH',
   'type': 'artist',
   'uri': 'spotify:artist:1AHswQqsDNmu1xaE8KpBne'}],
 'available_markets': ['AR',
  'AU',
  'AT',
  'BE',
  'BO',
  'BR',
  'BG',
  'CA',
  'CL',
  'CO',
  'CR',
  'CY',
  'CZ',
  'DK',
  'DO',
  'DE',
  'EC',
  'EE',
  'SV',
  'FI',
  'FR',
  'GR',
  'GT',
  'HN',
  'HK',
  'HU',
  'IS',
  'IE',
  'IT',
  'LV',
  'LT',
  'LU',
  'MY',
  'MT',
  'MX',
  'NL',
  'NZ',
  'NI',
  'NO',
  'PA',
  'PY',
  'PE',
  'PH',
  'PL',

In [105]:
list(my_track.album)

['album_type',
 'artists',
 'available_markets',
 'copyrights',
 'external_ids',
 'external_urls',
 'genres',
 'href',
 'id',
 'images',
 'label',
 'name',
 'popularity',
 'release_date',
 'release_date_precision',
 'total_tracks',
 'tracks',
 'type',
 'uri']

In [89]:
print(my_track)

"MAMI" by Chris Lorenzo


In [91]:
result['type']

'track'

# Issues

Recently played isn't returning records after a milisecond time stamp, I think it works more on a 12hr basis. Probably best to call this a couple or times per day

# Archive

### get all artists from streaming history

In [94]:
result_list = []
iteration = 0
start = datetime.now()
for track in streaming_data:
    iteration += 1
    artist_name = track['artistName']
    # track_name = track['trackName']
    # end_time = track['endTime']
    # ms_played = track['msPlayed']
    if artist_name not in result_list:
        result_list.append(artist_name)

In [ ]:
podcasts = ['My Brother, My Brother And Me',
 'The Empty Bowl',
 'Marketplace',
 'The Always Sunny Podcast',
 'The Besties',
 'This Is Important',
 'SmartLess',
 'The Journal.',
 'Whiskey Ginger with Andrew Santino',
 'Code Switch']

### Parse items old

In [ ]:
###
# runs through the results of recently played and flattens the dictionary
# returns a cleaned dictionary of recently played
###
def parse_recently_played(recently_played_item):
    result = recently_played_item['track']
    subset = dict((k, result[k]) for k in ('id', 'name', 'duration_ms', 'popularity', 'disc_number', 'track_number', 'explicit', 'type'))
    subset['album_id'] = result['album']['id']
    subset['artist_id'] = result['artists'][0]['id']
    subset['artist_count'] = len(result['artists'])
    subset['played_at'] = recently_played_item['played_at']
    return subset

### Datetime handling

In [26]:
utc = datetime.strptime(current_max_played, "%Y-%m-%dT%H:%M:%S.%fZ")
from_zone = tz.tzutc()
to_zone = tz.tzlocal()
utc = utc.replace(tzinfo=from_zone)
local = utc.astimezone(to_zone)
local_unix = int(time.mktime(local.timetuple())*1000)

### List Json File Names

In [16]:
path = "C://Users//Brian//Documents//python//spotify//MyData//"
dir_list = os.listdir(path)
print(dir_list)

['DuoNewFamily.json', 'Follow.json', 'Identifiers.json', 'Inferences.json', 'Marquee.json', 'Payments.json', 'Playlist1.json', 'Read_Me_First.pdf', 'SearchQueries.json', 'StreamingHistory0.json', 'UserAddress.json', 'Userdata.json', 'YourLibrary.json']


In [129]:
# Get the list of all files and directories
path = "C://Users//Brian//Documents//python//spotify//MyData//"
spotify_file_list = os.listdir(path)

# prints all files
for i in spotify_file_list:
    print(i.replace('.json', ''))

DuoNewFamily
Follow
Identifiers
Inferences
Marquee
Payments
Playlist1
Read_Me_First.pdf
SearchQueries
StreamingHistory0
UserAddress
Userdata
YourLibrary


### Simple Query From Spotipy Docs

In [6]:
scope = "user-library-read"

results = sp.current_user_saved_tracks()
for idx, item in enumerate(results['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], " – ", track['name'])

0 Slothrust  –  Pony
1 The Steeldrivers  –  East Kentucky Home
2 Norman Greenbaum  –  Spirit in the Sky
3 Redbone  –  Come and Get Your Love (Rerecorded Version)
4 Blue Swede  –  Hooked On A Feeling
5 Rupert Holmes  –  Escape (The Pina Colada Song)
6 Plague Vendor  –  Ox Blood
7 Art Tatum  –  Blue Skies
8 The Dillards  –  Dooley
9 Matroda  –  Gimme Some Keys
10 FJAAK  –  Plan of Escape
11 Chris Lorenzo  –  MAMI
12 Dreamville  –  Ma Boy
13 Ethereal  –  Beef
14 Freddie Gibbs  –  Couldn’t Be Done (feat. Kelly Price)
15 Quindon Tarver  –  Everybody's Free (To Wear Sunscreen)
16 Freddie Gibbs  –  Something to Rap About (feat. Tyler, The Creator)
17 Freddie Gibbs  –  Dark Hearted
18 Freddie Gibbs  –  PYS (feat. DJ Paul)
19 Freddie Gibbs  –  Feel No Pain (feat. Anderson .Paak & Raekwon)


### Playlist Info

In [ ]:
# print information about playlist names and IDs
# print(print_playlist_names(user_id))

def print_playlist_names(user_id):
    playlists = sp.user_playlists(user_id)
    while playlists:
        for i, playlist in enumerate(playlists['items']):
            print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
        if playlists['next']:
            playlists = sp.next(playlists)
        else:
            playlists = None
        


# Read JSON File Version

def get_playlist_data_old():
    playlist_data = get_json_from_file('Playlist1')['playlists']
    print('Number of playlists: {}'.format(len(playlist_data)))
    print('')
    print('List Playlist Names:')
    list_index = 0
    for i in playlist_data:
        print(i['name'],'index:',list_index)
        list_index += 1
    return playlist_data

In [111]:
playlists_data = get_playlist_data()

playlists_data[2]

Number of playlists: 42

List Playlist Names:
My recommendation playlist index: 0
Happy Birthday Mama! index: 1
Liked em index: 2
Listen List index: 3
Love songs index: 4
Karaoke Mix index: 5
Wedding index: 6
Dancing index: 7
Bacci Boys index: 8
Chilll index: 9
Summah index: 10
Funky index: 11
Random New index: 12
eclectric index: 13
Persian index: 14
Easy Listening index: 15
For My Love index: 16
nothiing index: 17
My Shazam Tracks index: 18
Pity Party index: 19
nate index: 20
you no index: 21
scRAP it index: 22
"Psychadelic" "Rock" index: 23
Pure Reggae Starbucks CD index: 24
Afro Cuban index: 25
FUN index: 26
Nold Scholl index: 27
House jammz index: 28
Glitch Hop index: 29
Yum Yum index: 30
Instrumentals index: 31
Ween index: 32
Folk index: 33
Jazz New index: 34
Rock Hard index: 35
Random discovered liked index: 36
Rock "Classics" index: 37
Lean on me index: 38
Rippitty Rap index: 39
Stoopid index: 40
Jazz Classic index: 41


{'name': 'Liked em',
 'lastModifiedDate': '2023-03-08',
 'items': [{'track': {'trackName': '151 Rum',
    'artistName': 'JID',
    'albumName': 'DiCaprio 2',
    'trackUri': 'spotify:track:22WV03i2lBbwNVCE1g671p'},
   'episode': None,
   'localTrack': None,
   'addedDate': '2023-02-07'},
  {'track': {'trackName': 'Two Tens (feat. Anderson .Paak)',
    'artistName': 'Cordae',
    'albumName': 'Two Tens (feat. Anderson .Paak)',
    'trackUri': 'spotify:track:6clDsO8HwhHEgJDDp88VdL'},
   'episode': None,
   'localTrack': None,
   'addedDate': '2023-02-07'},
  {'track': {'trackName': 'Ma Boy',
    'artistName': 'Dreamville',
    'albumName': 'Ma Boy',
    'trackUri': 'spotify:track:1kn9JsBWIw6qREhDWw0lKb'},
   'episode': None,
   'localTrack': None,
   'addedDate': '2023-02-07'},
  {'track': {'trackName': 'Eyes in the Back of My Head',
    'artistName': 'Katori Walker',
    'albumName': 'Eyes in the Back of My Head',
    'trackUri': 'spotify:track:5MrtQggewKtfSzs4akoaD2'},
   'episode': No

### Crap

In [5]:
get_most_recent_timestamp()

(1697629141000,
 datetime.datetime(2023, 10, 18, 7, 39, 1, 795000, tzinfo=tzlocal()))

In [66]:
print('BASIC ARTIST DATA')    
print(get_artist_data('1J6XFH9yaFImOmbaAWAx48'))
print('-----------------------------')
print('BASIC TRACK DATA')
print(get_track_data('7D5gkUVhkLbe5e8qG1NqcZ'))
print('-----------------------------')
print('BASIC ALBUM DATA')
print(get_album_data('4EAehCii5lZgeewct1LA5p'))

BASIC ARTIST DATA
{'id': '1J6XFH9yaFImOmbaAWAx48', 'name': 'R. Guru', 'genres': ['rain'], 'popularity': 53, 'followers': 460}
-----------------------------
BASIC TRACK DATA
{'id': '7D5gkUVhkLbe5e8qG1NqcZ', 'name': 'Brighter', 'duration_ms': 281084, 'popularity': 60, 'disc_number': 1, 'track_number': 1, 'explicit': False, 'type': 'track', 'album_id': '4EAehCii5lZgeewct1LA5p', 'artist_id': '5Pb27ujIyYb33zBqVysBkj', 'artist_count': 1}
-----------------------------
BASIC ALBUM DATA
{'id': '4EAehCii5lZgeewct1LA5p', 'name': 'Bloom', 'album_type': 'album', 'release_date': '2016-01-22', 'release_date_precision': 'day', 'popularity': 74, 'total_tracks': 11, 'genres': [], 'label': 'Sweat It Out', 'type': 'album'}
